In [9]:
# Import all neccessary libraries

from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import requests

In [10]:
# Assign the URL to a variable

url = "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=M1C&DGUIDlist=2021A0011M1C&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0"

# use the urlopen function to open the webpage
html = urllib.request.urlopen(url)

# show object html
html

In [11]:
# Create the BeautifulSoup object

html_to_parse = BeautifulSoup(html, "html.parser")

In [12]:
# create a list of tables.  There is only 1 table in this webpage

tables = html_to_parse.find_all("table")
print(f"Number of tables found: {len(tables)}")

Number of tables found: 1


In [13]:
toronto_DF = pd.read_csv('postalcode.csv')

In [15]:
import urllib.parse as urlparse
from urllib.parse import urlencode

In [19]:
def scrape_census_data(postal_codes, data_points):
    """
    Scrape multiple census data points and women's age data for given postal codes
    
    Parameters:
    postal_codes (list): List of postal codes to scrape
    data_points (dict): Dictionary of data points to collect, where key is column name
                       and value is the title text to search for
    
    Returns:
    pandas.DataFrame: DataFrame containing all collected data
    """
    
    # Define the age ranges we want to collect
    age_ranges = [
        "20 to 24 years - Counts - Women＋",
        "25 to 29 years - Counts - Women＋",
        "30 to 34 years - Counts - Women＋",
        "35 to 39 years - Counts - Women＋"
    ]
    
    # Create empty DataFrame with columns for regular data points and women's age data
    columns = ['postal_code'] + list(data_points.keys()) + ['women_20_39']
    df = pd.DataFrame(columns=columns)
    
    # Loop through each postal code
    for postal_code in postal_codes:
        print(f"\nProcessing postal code: {postal_code}")
        
        # Construct the URL for general data
        url = "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E"
        params = {
            'SearchText': postal_code,
            'DGUIDlist': f'2021A0011{postal_code}',
            'GENDERlist': '1,2,3',
            'STATISTIClist': '1',
            'HEADERlist': '0'
        }
        
        # Build the full URL
        url_parts = list(urlparse.urlparse(url))
        query = dict(urlparse.parse_qsl(url_parts[4]))
        query.update(params)
        url_parts[4] = urlencode(query)
        query = urlparse.urlunparse(url_parts)
        
        try:
            # Get the webpage content
            html = urllib.request.urlopen(query)
            html_to_parse = BeautifulSoup(html, "html.parser")
            tables = html_to_parse.find_all("table")
            
            if not tables:
                print(f"No tables found for postal code {postal_code}")
                continue
            
            # Initialize row data with postal code
            row_data = {'postal_code': postal_code}
            
            # Collect general data points
            for column_name, title_text in data_points.items():
                try:
                    # Find the cell with the specified title
                    title = f"2021A0011{postal_code} - {title_text}"
                    td = tables[0].find(attrs={"title": title})
                    
                    if td:
                        # Convert the value to float, removing commas and $ signs
                        value = td.text.strip().replace(",", "").replace("$", "")
                        # Handle percentage values
                        if "%" in value:
                            value = value.replace("%", "")
                        value = float(value)
                        row_data[column_name] = value
                        print(f"Found {column_name}: {value}")
                    else:
                        print(f"Data point '{title_text}' not found for {postal_code}")
                        row_data[column_name] = None
                        
                except Exception as e:
                    print(f"Error processing {title_text} for {postal_code}: {str(e)}")
                    row_data[column_name] = None
            
            # Collect women's age data
            women_sum = 0
            for age_range in age_ranges:
                try:
                    # Find the cell with the specified title for women's count
                    td = tables[0].find(
                        "td",
                        attrs={
                            "headers": lambda x: x and f"geo2021A0011{postal_code}stat1gen3" in x,
                            "title": lambda x: x and age_range in x
                        }
                    )
                    
                    if td:
                        value = float(td.text.strip().replace(",", ""))
                        women_sum += value
                        print(f"Found {value} women for {age_range}")
                    else:
                        print(f"Age range '{age_range}' not found")
                        
                except Exception as e:
                    print(f"Error processing {age_range}: {str(e)}")
            
            row_data['women_20_39'] = women_sum
            
            # Add the row to the DataFrame
            df.loc[len(df.index)] = row_data
            
        except Exception as e:
            print(f"Error processing postal code {postal_code}: {str(e)}")
            continue
    
    return df


In [23]:
# Define the data points to collect
census_data_points = {
    'total_population': 'Population, 2021 - Counts - Total',
    'average_income_women': 'Average total income in 2020 among recipients ($) - Counts - Women＋',
    'average_age_women': 'Average age of the population - Counts - Women＋'
}

# Get postal codes from your CSV file
postal_codes = toronto_DF['Postalcode'].tolist()

# Run the scraper
census_df = scrape_census_data(postal_codes, census_data_points)  # Testing with first 5 postal codes


Processing postal code: M1B
Found total_population: 65555.0
Found average_income_women: 38360.0
Found average_age_women: 41.9
Found 2390.0 women for 20 to 24 years - Counts - Women＋
Found 2575.0 women for 25 to 29 years - Counts - Women＋
Found 2355.0 women for 30 to 34 years - Counts - Women＋
Found 2140.0 women for 35 to 39 years - Counts - Women＋

Processing postal code: M1C
Found total_population: 35642.0
Found average_income_women: 52100.0
Found average_age_women: 44.3
Found 1195.0 women for 20 to 24 years - Counts - Women＋
Found 1055.0 women for 25 to 29 years - Counts - Women＋
Found 900.0 women for 30 to 34 years - Counts - Women＋
Found 1020.0 women for 35 to 39 years - Counts - Women＋

Processing postal code: M1E
Found total_population: 48033.0
Found average_income_women: 42320.0
Found average_age_women: 43.1
Found 1635.0 women for 20 to 24 years - Counts - Women＋
Found 1620.0 women for 25 to 29 years - Counts - Women＋
Found 1555.0 women for 30 to 34 years - Counts - Women＋
Foun

/tmp/ipykernel_52/819404319.py:110: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df.loc[len(df.index)] = row_data


No tables found for postal code M7R

Processing postal code: M7Y
No tables found for postal code M7Y

Processing postal code: M8V
Found total_population: 44144.0
Found average_income_women: 59200.0
Found average_age_women: 43.0
Found 1150.0 women for 20 to 24 years - Counts - Women＋
Found 2205.0 women for 25 to 29 years - Counts - Women＋
Found 2700.0 women for 30 to 34 years - Counts - Women＋
Found 2110.0 women for 35 to 39 years - Counts - Women＋

Processing postal code: M8W
Found total_population: 22381.0
Found average_income_women: 55150.0
Found average_age_women: 42.6
Found 520.0 women for 20 to 24 years - Counts - Women＋
Found 725.0 women for 25 to 29 years - Counts - Women＋
Found 1015.0 women for 30 to 34 years - Counts - Women＋
Found 950.0 women for 35 to 39 years - Counts - Women＋

Processing postal code: M8X
Found total_population: 10624.0
Found average_income_women: 85200.0
Found average_age_women: 49.3
Found 250.0 women for 20 to 24 years - Counts - Women＋
Found 215.0 women 

In [24]:
census_df

,postal_code,total_population,average_income_women,average_age_women,women_20_39
0,M1B,65555.0,38360.0,41.9,9460.0
1,M1C,35642.0,52100.0,44.3,4170.0
2,M1E,48033.0,42320.0,43.1,6335.0
3,M1G,30894.0,35720.0,40.3,4595.0
4,M1H,23964.0,38400.0,42.1,3785.0
...,...,...,...,...,...
92,M9N,26059.0,40960.0,41.4,3795.0
93,M9P,20645.0,50320.0,46.5,2505.0
94,M9R,33783.0,43000.0,43.2,4395.0
95,M9V,53878.0,34840.0,40.2,7975.0


In [32]:
census_df = census_df.dropna()
census_df

,postal_code,total_population,average_income_women,average_age_women,women_20_39
0,M1B,65555.0,38360.0,41.9,9460.0
1,M1C,35642.0,52100.0,44.3,4170.0
2,M1E,48033.0,42320.0,43.1,6335.0
3,M1G,30894.0,35720.0,40.3,4595.0
4,M1H,23964.0,38400.0,42.1,3785.0
...,...,...,...,...,...
92,M9N,26059.0,40960.0,41.4,3795.0
93,M9P,20645.0,50320.0,46.5,2505.0
94,M9R,33783.0,43000.0,43.2,4395.0
95,M9V,53878.0,34840.0,40.2,7975.0


In [33]:
census_df.to_csv('census.csv',index=False)